In [1]:
from collections import Counter, defaultdict
import sys
from typing import Tuple


import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from seqeval import metrics
import torch
import transformers as tfm

sys.path.append("../src")

import ner

%load_ext autoreload
%autoreload 2


/Users/spayot/miniforge3/envs/tfm4/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', None)

MODEL_CKPT = "xlm-roberta-base"

# MultiLingual NER
* Using XLM-R model
* task: create NER model for swiss languages (de, fr, it, en)
* dataset: XTREME - PAN-X: de, fr, it, en

## Dataset
### Load

In [3]:
langs = {"de": 0.629, "fr": 0.229, "it": 0.084, "en": 0.059}

# return a DatasetDict if a key doesn't exist
panx_ch = defaultdict(datasets.DatasetDict)

for lang, frac in langs.items():
    # load monolingual corpus
    ds = datasets.load_dataset("xtreme", name=f"PAN-X.{lang}")
    # sample examples to create imbalanced dataset based on language prevalence
    for split in ds:
        panx_ch[lang][split] = (ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

    

Found cached dataset xtreme (/Users/spayot/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)
100%|██████████| 3/3 [00:00<00:00, 265.79it/s]
Loading cached shuffled indices for dataset at /Users/spayot/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-e5ddf09f1ae095ec.arrow
Loading cached shuffled indices for dataset at /Users/spayot/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-25e7e2dd003d0fa6.arrow
Loading cached shuffled indices for dataset at /Users/spayot/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-73a95bc0accfea8b.arrow
Found cached dataset xtreme (/Users/spayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)
100%|██████████| 3/3

### Exploratory Data Analysis

In [4]:
# inspecting the number of examples for each language
for lang, ds in panx_ch.items():
    print(lang, ds["train"].num_rows, "rows", sep=" ")

de 12580 rows
fr 4580 rows
it 1680 rows
en 1180 rows


In [5]:
# what are the features in each dataset?
panx_ch["de"]["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [6]:
# let's add string tags 
tags = panx_ch["de"]["train"].features["ner_tags"].feature

def create_tag_names(batch: dict):
    # tags.int2str turns a list of ids into str labels
    tag_names: list = [tags.int2str(tag_id) for tag_id in batch["ner_tags"]] 
    
    return {"ner_tags_str": tag_names}

batch = ds["train"][:3]

print(create_tag_names(batch))

    

{'ner_tags_str': [['O', 'O', 'B-ORG', 'I-ORG', 'O', 'O'], ['B-ORG', 'I-ORG', 'O', 'B-PER', 'I-PER'], ['B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O']]}


In [7]:
panx_de = panx_ch["de"].map(create_tag_names)

de_example = panx_de["train"][0]

pd.DataFrame(de_example)[["tokens", "ner_tags_str"]].T



100%|██████████| 6290/6290 [00:00<00:00, 19330.94ex/s]


,0,1,2,3,4,5,6,7,8,9,10,11
tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
ner_tags_str,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


#### How many of each tags in DE dataset?

In [8]:
freqs = defaultdict(Counter)

for split, ds in panx_de.items():
    for tag_list in ds["ner_tags_str"]:
        for tag in tag_list:
            if tag.startswith("B-"):
                freqs[split][tag[2:]] += 1

pd.DataFrame(freqs)

,train,validation,test
LOC,6186,3172,3180
ORG,5366,2683,2573
PER,5810,2893,3071


## Tokenizer

In [9]:
xlmr_tokenizer = tfm.AutoTokenizer.from_pretrained(MODEL_CKPT)

text = "This can probably be done. N'est-ce pas?"

print(xlmr_tokenizer(text).tokens())

Downloading: 100%|██████████| 615/615 [00:00<00:00, 421kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:00<00:00, 5.48MB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:01<00:00, 7.06MB/s]


['<s>', '▁This', '▁can', '▁probably', '▁be', '▁done', '.', '▁N', "'", 'est', '-', 'ce', '▁pas', '?', '</s>']


## Model instantiation

In [89]:
id2label = dict(enumerate(tags.names))
label2id = {tag: idx for idx, tag in enumerate(tags.names)}

xlmr_config = tfm.AutoConfig.from_pretrained(MODEL_CKPT, num_labels=tags.num_classes, 
                                         id2label=id2label, label2id=label2id)

xlmr = tfm.AutoModelForTokenClassification.from_pretrained(MODEL_CKPT, config = xlmr_config)

In [92]:
# verify that the model outputs something with the expected format:
print(f"original text \"{text}\"")

tokens = xlmr_tokenizer(text).tokens()
print(f"number of tokens after tokenization: {len(xlmr_tokenizer(text).tokens())}")

input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")

outputs = xlmr(input_ids)
print(f"outputs size: {outputs.logits.size()}")

pred_ids = outputs.logits.argmax(-1).numpy()
example = {
    "tokens": tokens, 
    "ids": input_ids[0].numpy(),
    "preds": outputs.logits.argmax(-1).numpy()[0],
    }

example = pd.DataFrame(example)

example["pred_labels"] = example.preds.map(id2label)
display(example.drop("preds", axis=1).T)

original text "This can probably be done. N'est-ce pas?"
number of tokens after tokenization: 15
outputs size: torch.Size([1, 15, 7])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,<s>,▁This,▁can,▁probably,▁be,▁done,.,▁N,',est,-,ce,▁pas,?,</s>
ids,0,3293,831,31895,186,16940,5,541,25,525,9,329,452,32,2
pred_labels,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,B-LOC,I-LOC,I-LOC,I-LOC


In [93]:
# refactored: turning above into a function
ner.inspect.tag_text(
    text="let's try with another text now. en francais", 
    model=xlmr,
    tokenizer=xlmr_tokenizer,
    tags=tags,
    )


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
tokens,<s>,▁let,',s,▁try,▁with,▁another,▁text,▁now,.,▁en,▁franca,is,</s>
tags,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,B-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC


## Preparing the Dataset
* tokenize doc
* add labels for unlabeled tokens (e.g. second part of split words, or initial character).

In [94]:
# refactored: moved tokenize_and_align_labels to ner.preproc module
def encode_panx_dataset(corpus: datasets.DatasetDict) -> datasets.DatasetDict:
    return corpus.map(ner.preproc.tokenize_and_align_labels,
        fn_kwargs={"tokenizer": xlmr_tokenizer}, 
        batched=True, 
        remove_columns=["langs", "ner_tags", "tokens"]
        )

panx_de_encoded = encode_panx_dataset(panx_ch["de"])

100%|██████████| 7/7 [00:00<00:00, 28.54ba/s]


In [95]:
example = panx_de_encoded["train"][0]
df = pd.DataFrame(example)
df["tokens"] = xlmr_tokenizer.convert_ids_to_tokens(example["input_ids"])
df.T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
input_ids,0,70101,176581,19,142,122,2290,708,1505,18363,...,13787,14,15263,18917,663,6947,19,6,5,2
attention_mask,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
labels,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


## Performance Measures
using seqeval library

In [96]:
# quick look at how seqeval metrics work.
y_true = [["B-a","B-b","I-b","I-b","B-c","B-c"]]
y_pred = [["B-a","B-b","I-b","B-a","B-c","B-c"]]

print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           a       0.50      1.00      0.67         1
           b       0.00      0.00      0.00         1
           c       1.00      1.00      1.00         2

   micro avg       0.60      0.75      0.67         4
   macro avg       0.50      0.67      0.56         4
weighted avg       0.62      0.75      0.67         4



In [100]:
# testing SeqevalConverter: 
seqeval_converter = ner.evaluate.SeqevalConverter(id2label)

test_preds = torch.rand((2,9,7)).numpy()
test_true = test_preds.argmax(-1)
test_true[0,0] = -100
test_true[1,0] = -100
test_true[0,-1] = -100
test_true[1,-1] = -100

test_true[0,4] = 4
test_true[1,4] = 4

test_true[0,5] = -100

y_true, y_pred = seqeval_converter.align_predictions(predictions=test_preds, label_ids=test_true)
print(y_true, y_pred, sep="\n")

eval_pred = tfm.EvalPrediction(test_preds, test_true)
seqeval_converter.compute_metrics(eval_pred)

[['B-ORG', 'B-PER', 'B-PER', 'I-ORG', 'I-LOC', 'B-LOC'], ['I-PER', 'B-LOC', 'B-PER', 'I-ORG', 'O', 'I-PER', 'I-PER']]
[['B-ORG', 'B-PER', 'B-PER', 'I-ORG', 'I-LOC', 'B-LOC'], ['I-PER', 'B-LOC', 'B-PER', 'B-PER', 'O', 'I-PER', 'I-PER']]


{'f1': 0.9090909090909091}

# FineTuning XLM-R

In [101]:
# define training arguments
BATCH_SIZE = 24
OUTPUT_DIR = f"../models/{MODEL_CKPT}-finetuned-panx-de"

training_config = {
    "output_dir": OUTPUT_DIR,
    "num_train_epochs": 3,
    "learning_rate": 2e-5,
    "per_device_train_batch_size": BATCH_SIZE,
    "per_device_eval_batch_size": BATCH_SIZE,
    "weight_decay": 0.01,
    "save_steps": 1e6,
    "evaluation_strategy": "epoch",
    "disable_tqdm": False,
    "logging_steps": len(panx_de_encoded["train"]) // BATCH_SIZE,
    "push_to_hub": False,
    "log_level": "error",
}

training_args = tfm.TrainingArguments(**training_config)

# refactored
seqeval_converter = ner.evaluate.SeqevalConverter(id2label)


# define data collation
data_collator = tfm.DataCollatorForTokenClassification(xlmr_tokenizer)

def model_init():
    return (tfm.AutoModelForTokenClassification.from_pretrained(MODEL_CKPT, config=xlmr_config))

trainer = tfm.Trainer(
    model_init=model_init, 
    tokenizer=xlmr_tokenizer,
    args=training_args,
    data_collator=data_collator, 
    compute_metrics=seqeval_converter.compute_metrics,
    train_dataset=panx_de_encoded["train"],
    eval_dataset=panx_de_encoded["validation"],
)


In [102]:
trainer.train()

/Users/spayot/miniforge3/envs/tfm4/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 33%|███▎      | 524/1575 [18:01<45:57,  2.62s/it]  

{'loss': 0.2962, 'learning_rate': 1.3346031746031746e-05, 'epoch': 1.0}


                                                  
 33%|███▎      | 525/1575 [19:24<39:35,  2.26s/it]

{'eval_loss': 0.16917505860328674, 'eval_f1': 0.8154074751400161, 'eval_runtime': 82.0346, 'eval_samples_per_second': 76.675, 'eval_steps_per_second': 3.206, 'epoch': 1.0}


 67%|██████▋   | 1048/1575 [38:24<14:54,  1.70s/it] 

...

100%|█████████▉| 1572/1575 [58:30<00:07,  2.55s/it]  

{'loss': 0.1097, 'learning_rate': 3.8095238095238096e-08, 'epoch': 2.99}


                                                   
100%|██████████| 1575/1575 [59:51<00:00,  2.28s/it]

{'eval_loss': 0.14076970517635345, 'eval_f1': 0.8597321174123681, 'eval_runtime': 75.7327, 'eval_samples_per_second': 83.055, 'eval_steps_per_second': 3.473, 'epoch': 3.0}
{'train_runtime': 3591.5774, 'train_samples_per_second': 10.508, 'train_steps_per_second': 0.439, 'train_loss': 0.18437748816278246, 'epoch': 3.0}


TrainOutput(global_step=1575, training_loss=0.18437748816278246, metrics={'train_runtime': 3591.5774, 'train_samples_per_second': 10.508, 'train_steps_per_second': 0.439, 'train_loss': 0.18437748816278246, 'epoch': 3.0})

In [103]:
trainer.evaluate(panx_de_encoded["validation"])

100%|██████████| 263/263 [01:16<00:00,  3.45it/s]


{'eval_loss': 0.14076970517635345,
 'eval_f1': 0.8597321174123681,
 'eval_runtime': 76.6659,
 'eval_samples_per_second': 82.044,
 'eval_steps_per_second': 3.43,
 'epoch': 3.0}

In [104]:
trainer.save_model()

#### Test trained model on a simple sentence

In [105]:
text_de = "Sylvain Payot ist ein Informatiker bei Nike in Kalifornien"
ner.inspect.tag_text(text_de, trainer.model, xlmr_tokenizer, tags)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,<s>,▁Sylv,ain,▁Pay,ot,▁ist,▁ein,▁Informati,ker,▁bei,▁Nike,▁in,▁Kaliforni,en,</s>
tags,O,B-PER,I-PER,I-PER,I-PER,O,O,O,O,O,B-ORG,O,B-LOC,I-LOC,O


In [106]:
# also works in French!
text_fr = "Sylvain Payot est un informaticien travaillant à Nike en Bretagne."
ner.inspect.tag_text(text_fr, trainer.model, xlmr_tokenizer, tags)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
tokens,<s>,▁Sylv,ain,▁Pay,ot,▁est,▁un,▁informatic,ien,▁travail,lant,▁à,▁Nike,▁en,▁,Bretagne,.,</s>
tags,O,B-PER,B-PER,I-PER,I-PER,O,O,O,O,O,O,O,B-ORG,O,B-LOC,B-LOC,O,O


### Error Analysis

In [153]:
# refactored
builder = ner.erroranalysis.TokensLossDataFrameBuilder(
    trainer=trainer,
    data_collator=data_collator,
)

In [156]:
# below is equivalent to analyzer.build_tokens_dataframe(<dataset>)
ds = builder._calculate_token_level_loss_on_dataset(panx_de_encoded["validation"])
df = builder._convert_dataset_to_pandas(ds)
df_tokens = builder._convert_to_token_level_dataframe(df)
df_tokens.head()

,input_ids,attention_mask,labels,loss,predicted_labels,input_tokens
0,10699,1,B-ORG,0.05,B-ORG,▁Ham
0,15,1,I-ORG,0.06,I-ORG,▁(
0,16104,1,I-ORG,0.05,I-ORG,▁Unternehmen
0,1388,1,I-ORG,0.06,I-ORG,▁)
1,56530,1,O,0.00,O,▁WE


In [157]:
# refactored
analyzer = ner.erroranalysis.NERLossAnalyzer(df_tokens)

display(analyzer.loss_groupby_analysis(groupby="input_tokens", sort="sum", ascending=False))
display(analyzer.loss_groupby_analysis(groupby="input_tokens", ascending=True))
display(analyzer.loss_groupby_analysis(groupby="input_tokens", sort="count", ascending=False))

,0,1,2,3,4,5,6,7,8,9
input_tokens,▁,▁von,▁in,▁der,▁/,▁und,▁(,▁),▁'',▁A
count,6066,808,989,1388,163,1171,246,246,2898,125
mean,0.04,0.17,0.13,0.08,0.56,0.07,0.32,0.31,0.02,0.42
sum,213.37,139.38,128.83,111.32,91.12,82.69,79.6,75.17,70.3,52.94


,0,1,2,3,4,5,6,7,8,9
input_tokens,▁−,▁Geburt,▁Geburtstag,▁Gebäude,▁Gedanken,▁Noch,▁Gefühl,▁Gegensatz,▁Gegner,▁Gebrauch
count,3,6,6,5,2,3,1,2,7,1
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9
input_tokens,▁,▁'',▁',▁der,▁und,▁in,▁die,▁von,▁er,▁den
count,6066,2898,2133,1388,1171,989,860,808,707,489
mean,0.04,0.02,0.02,0.08,0.07,0.13,0.05,0.17,0.0,0.03
sum,213.37,70.3,34.46,111.32,82.69,128.83,42.8,139.38,0.0,13.42


#### Visualize the Confusion Matrix

In [158]:
ner.plot.plot_confusion_matrix(df_tokens.labels, df_tokens.predicted_labels, tags.names)

## Evaluate Cross Lingual Performance
* evaluating performance on other languages without re-training.

In [ ]:
def get_f1_score(trainer: transformers.Trainer, dataset: datasets.Dataset) -> float:
    return trainer.predict(dataset).metrics["test_f1"]


f1_scores = defaultdict(dict)
f1_scores["de"]["de"] = get_f1_score(trainer, panx_de_encoded["test"])
print(f"f1-scores of [de]-trained model on [de] test dataset is: {f1_scores['de']['de']:.3f}")

In [ ]:
def evaluate_lang_performance(lang: str, trainer: transformers.Trainer) -> float:
    ds_encoded = encode_panx_dataset(panx_ch[lang])
    return get_f1_score(trainer, ds_encoded["test"])

for lang in ["fr", "it", "en"]:
    f1_scores["de"][lang] = evaluate_lang_performance(lang, trainer)
    print(f"f1-score of [de]-trained model on [{lang}] test dataset is: {f1_scores['de'][lang]:.3f}")

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

f1-score of [de]-trained model on fr test dataset is: 0.706


  0%|          | 0/2 [00:00<?, ?ba/s]

...

f1-score of [de]-trained model on it test dataset is: 0.696


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

f1-score of [de]-trained model on en test dataset is: 0.598


### what language specific dataset size is necessary to beat zero-shot transfer on higher resource model?
* example from FR
steps:
* train a model on a dataset of increasing size in French
* evaluate its performance on fr test dataset


In [ ]:
def train_on_subset(dataset: datasets.Dataset, num_samples: int) -> pd.DataFrame:
    """train a model on a subset of a total dataset of size `num_samples`. extract f1-scores"""

    train_ds = dataset["train"].shuffle(seed=42).select(range(num_samples))
    valid_ds = dataset["validation"]
    test_ds = dataset["test"]

    training_args.logging_steps = len(train_ds) // BATCH_SIZE
    trainer = transformers.Trainer(
        model_init=model_init, 
        args=training_args,
        data_collator=data_collator, 
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        tokenizer=xlmr_tokenizer,
    )

    trainer.train()

    f1_score = get_f1_score(trainer=trainer, dataset=test_ds)

    return pd.DataFrame({"num_samples": [num_samples], "f1_score": [f1_score]})



In [ ]:
panx_fr_encoded = encode_panx_dataset(panx_ch["fr"])

subset_df = train_on_subset(panx_fr_encoded, 250)
subset_df

  0%|          | 0/5 [00:00<?, ?ba/s]

Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-d94243564fc5fbf9.arrow
Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-05980361e32c5986.arrow
/opt/homebrew/Caskroom/miniforge/base/envs/transformers_test/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/33 [00:00<?, ?it/s]

{'loss': 2.0912, 'learning_rate': 1.3939393939393942e-05, 'epoch': 0.91}


  0%|          | 0/96 [00:00<?, ?it/s]

...

{'eval_loss': 1.3672953844070435, 'eval_f1': 0.024362215582624684, 'eval_runtime': 22.2637, 'eval_samples_per_second': 102.858, 'eval_steps_per_second': 4.312, 'epoch': 2.0}
{'loss': 1.3766, 'learning_rate': 1.8181818181818183e-06, 'epoch': 2.73}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 1.3202787637710571, 'eval_f1': 0.06296370563465009, 'eval_runtime': 23.5531, 'eval_samples_per_second': 97.227, 'eval_steps_per_second': 4.076, 'epoch': 3.0}
{'train_runtime': 181.6237, 'train_samples_per_second': 4.129, 'train_steps_per_second': 0.182, 'train_loss': 1.634277105331421, 'epoch': 3.0}


  0%|          | 0/96 [00:00<?, ?it/s]

,num_samples,f1_score
0,250,0.062239


In [ ]:
for num_samples in [500, 1000, 2000]:
    subset_df = subset_df.append(train_on_subset(panx_fr_encoded, num_samples), ignore_index=True)

Loading cached shuffled indices for dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-e0b77cda78ac93c9.arrow
/opt/homebrew/Caskroom/miniforge/base/envs/transformers_test/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/63 [00:00<?, ?it/s]

{'loss': 1.7986, 'learning_rate': 1.3650793650793652e-05, 'epoch': 0.95}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 1.307023048400879, 'eval_f1': 0.050548995235135695, 'eval_runtime': 23.0622, 'eval_samples_per_second': 99.297, 'eval_steps_per_second': 4.163, 'epoch': 1.0}
{'loss': 1.25, 'learning_rate': 7.301587301587301e-06, 'epoch': 1.9}


...

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.38927942514419556, 'eval_f1': 0.756385068762279, 'eval_runtime': 21.3099, 'eval_samples_per_second': 107.462, 'eval_steps_per_second': 4.505, 'epoch': 2.0}
{'loss': 0.3266, 'learning_rate': 2.3809523809523811e-07, 'epoch': 2.96}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.3719795346260071, 'eval_f1': 0.7756653992395438, 'eval_runtime': 22.559, 'eval_samples_per_second': 101.511, 'eval_steps_per_second': 4.256, 'epoch': 3.0}
{'train_runtime': 762.295, 'train_samples_per_second': 7.871, 'train_steps_per_second': 0.331, 'train_loss': 0.6466079997637916, 'epoch': 3.0}


  0%|          | 0/96 [00:00<?, ?it/s]

In [ ]:
subset_df

,num_samples,f1_score
0,250,0.062239
1,500,0.204258
2,1000,0.665959
3,2000,0.764503


In [ ]:
fig, ax = plt.subplots()
ax.axhline(f1_scores["de"]["fr"], ls="--", color="r")
subset_df.set_index("num_samples").plot(ax=ax)
plt.legend(["Zero-shot from [de]", "Fine-tuned on [fr]"], loc="lower right")
plt.ylim((0,1))
plt.xlabel("Number of Training Samples")
plt.ylabel("F1-Score")
plt.show()

## Training on all languages at once
### Encode and Concatenate datasets for each language

In [ ]:
def concatenate_splits(corpora: list[datasets.DatasetDict]) -> datasets.DatasetDict:
    multi_corpus = datasets.DatasetDict()
    for split in corpora[0].keys():
        multi_corpus[split] = datasets.concatenate_datasets(
            [corpus[split] for corpus in corpora]
            ).shuffle(seed=42)
    return multi_corpus


langs = ["de", "fr", "it", "en"]

# initialize corpora
corpora = [panx_de_encoded]

corpora += [encode_panx_dataset(panx_ch[lang]) for lang in langs[1:]]

corpora_encoded = concatenate_splits(corpora)



Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-07fa17e04475672f.arrow
Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-07dac438cc574410.arrow
Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-66fe4a154cc188a1.arrow
Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-407e926815bd48fc.arrow
Loading cached processed dataset at /Users/SPayot/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/349258adc25bb45e47de193222f95e68a44f7a7ab53c4283b3f007208a11bf7e/cache-dd65a58d0ac05588.arrow
Loading cached processed datas

In [ ]:
for split in corpora_encoded:
    print(f"{split:<20} {len(corpora_encoded[split])}")

train                20020
validation           10010
test                 10010


In [ ]:
training_args.logging_steps = len(corpora_encoded["train"]) // BATCH_SIZE
training_args.output_dir = "xlm-roberta-base-finetuned-panx-all-ch"

xtrainer = tfm.Trainer(
    model_init=model_init, 
    args=training_args,
    data_collator=data_collator, 
    compute_metrics=compute_metrics,
    train_dataset=corpora_encoded["train"],
    eval_dataset=corpora_encoded["validation"],
    tokenizer=xlmr_tokenizer,
)

xtrainer.train()

for idx, lang in enumerate(langs):
    f1_scores["all"][lang] = get_f1_score(xtrainer, corpora[idx]["test"])

print(f1_scores)

/opt/homebrew/Caskroom/miniforge/base/envs/transformers_test/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2505 [00:00<?, ?it/s]

{'loss': 0.3333, 'learning_rate': 1.3341317365269462e-05, 'epoch': 1.0}


  0%|          | 0/418 [00:00<?, ?it/s]

{'eval_loss': 0.19712050259113312, 'eval_f1': 0.812826620214014, 'eval_runtime': 115.135, 'eval_samples_per_second': 86.941, 'eval_steps_per_second': 3.631, 'epoch': 1.0}
{'loss': 0.1713, 'learning_rate': 6.682634730538923e-06, 'epoch': 2.0}


...

{'eval_loss': 0.17813758552074432, 'eval_f1': 0.8376906710901159, 'eval_runtime': 114.0085, 'eval_samples_per_second': 87.801, 'eval_steps_per_second': 3.666, 'epoch': 2.0}
{'loss': 0.1312, 'learning_rate': 2.3952095808383236e-08, 'epoch': 3.0}


  0%|          | 0/418 [00:00<?, ?it/s]

{'eval_loss': 0.17378000915050507, 'eval_f1': 0.8485569985569985, 'eval_runtime': 115.7807, 'eval_samples_per_second': 86.457, 'eval_steps_per_second': 3.61, 'epoch': 3.0}
{'train_runtime': 7077.6999, 'train_samples_per_second': 8.486, 'train_steps_per_second': 0.354, 'train_loss': 0.2118049201613177, 'epoch': 3.0}


  0%|          | 0/263 [00:00<?, ?it/s]

defaultdict(<class 'dict'>, {'all': {'de': 0.8580619673796489, 'fr': 0.8601002749474366, 'it': 0.8642732049036776, 'en': 0.7473684210526317}})


## Save Model

In [ ]:
# saves in args.output_dir directory
xtrainer.save_model()

## Reload a model

In [11]:
NEW_MODEL = "../models/xlm-roberta-base-finetuned-panx-all-ch"

text = "Sylvain Payot est un data scientist qui travaille en Bretagne pour Nike"
ner = tfm.pipeline("ner", model=NEW_MODEL)
ner(text)



[{'entity': 'B-PER',
  'score': 0.9855789,
  'index': 1,
  'word': '▁Sylv',
  'start': 0,
  'end': 4},
 {'entity': 'I-PER',
  'score': 0.924334,
  'index': 2,
  'word': 'ain',
  'start': 4,
  'end': 7},
 {'entity': 'I-PER',
  'score': 0.99736816,
  'index': 3,
  'word': '▁Pay',
  'start': 8,
  'end': 11},
 {'entity': 'I-PER',
  'score': 0.9969472,
  'index': 4,
  'word': 'ot',
  'start': 11,
  'end': 13},
 {'entity': 'B-LOC',
  'score': 0.7727065,
  'index': 12,
  'word': '▁',
  'start': 53,
  'end': 54},
 {'entity': 'B-LOC',
  'score': 0.983724,
  'index': 13,
  'word': 'Bretagne',
  'start': 53,
  'end': 61},
 {'entity': 'B-ORG',
  'score': 0.93293,
  'index': 15,
  'word': '▁Nike',
  'start': 67,
  'end': 71}]

In [5]:
NEW_MODEL = "xlm-roberta-base-finetuned-panx-all-ch"
model = tfm.AutoModelForTokenClassification.from_pretrained(NEW_MODEL)

n_params = sum(p.numel() for p in model.parameters())
print(f"number of parameters: {n_params:,}")

trainer = tfm.Trainer(
    model=model, 
    args=training_args,
    data_collator=data_collator, 
    compute_metrics=compute_metrics,
    train_dataset=corpora_encoded["train"],
    eval_dataset=corpora_encoded["validation"],
    tokenizer=xlmr_tokenizer,
)
trainer.evaluate(panx_de_encoded["validation"])

In [82]:
ner.inspect.tag_text("Sylvain Payot est un data scientist qui travaille en Bretagne pour Nike", xtrainer.model, xlmr_tokenizer, tags)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
tokens,<s>,▁Sylv,ain,▁Pay,ot,▁est,▁un,▁data,▁scientist,▁qui,▁travaille,▁en,▁,Bretagne,▁pour,▁Nike,</s>
tags,I-ORG,B-PER,I-PER,I-PER,I-PER,O,O,O,O,O,O,O,B-LOC,B-LOC,O,B-ORG,O
